In [7]:
import os
os.environ["OPENAI_API_KEY"] = 'your_api_key'

In [4]:
from langchain.vectorstores import Chroma, ElasticKnnSearch
from langchain.embeddings import OpenAIEmbeddings, cohere as ce
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, CharacterTextSplitter
)
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, ConversationChain
from langchain.document_loaders import (
    TextLoader, DirectoryLoader, PyPDFLoader
)
from langchain.prompts import (
    PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
)
from langchain.chat_models import ChatOpenAI
from langchain.memory import (
    ChatMessageHistory, ConversationBufferMemory, ConversationBufferWindowMemory
)
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import MessagesPlaceholder
import random

In [5]:
# PyPDFLoader를 사용하여 'investment.pdf' 파일을 로드합니다.
loader = PyPDFLoader('/content/drive/MyDrive/investment.pdf')
# 로더를 사용하여 PDF 파일의 내용을 documents 변수에 로드합니다.
documents = loader.load()
# 'CharacterTextSplitter'를 사용하여 텍스트를 크기 1000의 청크로 분할합니다. 청크 간의 겹침은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
# 분할자를 사용하여 'documents'를 청크로 분할하고 결과를 texts 변수에 저장합니다.
texts = text_splitter.split_documents(documents)

In [8]:
#벡터 데이터베이스가 저장될 디렉토리의 이름을 변수에 저장합니다.
persist_directory = 'db'
# 텍스트 문서의 임베딩을 생성하는데 사용할 모델을 변수에 할당합니다.
embedding = OpenAIEmbeddings()
# Chroma 클래스의 'from_documents' 메서드를 호출하여 문서의 임베딩을 생성하고 벡터데이터 베이스를 초기화합니다.
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory)

In [9]:
# persist 메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 디스크에 저장합니다.
# 이렇게 하면 나중에 데이터베이스를 다시 로드하여 사용할 수 있습니다.
vectordb.persist()
# 메모리에서 데이터베이스를 제거합니다.
vectordb = None
# Chroma 클래스를 인스턴스화하여 vectordb 변수에 할당합니다. 문서의 임베딩을 저장하고 검색하는데 사용됩니다.
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding)

In [10]:
# 'as_retriever'메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 검색가능한 형태로 변환합니다.
retriever = vectordb.as_retriever()
# 질문과 관련된 문서를 검색합니다.(기본 k값은 4로 설정되어있어, 관련도가 높은 4개의 문서를 검색합니다.)
docs = retriever.get_relevant_documents("2023년 상반기 금리는어떘어?")
# 각 문서의 페이지 번호를 가져와 출력합니다.
for doc in docs:
    print(doc.metadata["page"])

22
30
32
31


## Suggested Question Generator

In [77]:
# Suggested Question Generator

prompt_template_generator = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
you are competent financial advisor. You need to create a suggested question,

{context}

Question: {question}

Don't answer with short answers, answer with complete sentences.
Answer in Korean:"""
# PROMPT에 'context'와 'question'을 입력 변수로하는 프롬프트 템플릿을 저장합니다.
PROMPT_generator = PromptTemplate(
    template=prompt_template_generator, input_variables=["context", "question"]
)
# 'chain_type_kwargs'변수에 딕셔너리를 저장합니다.
chain_type_kwargs_generator = {"prompt": PROMPT_generator}
# 'as_retriever'메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 검색 가능한 형태로 변환합니다.
# 유사도가 높은 상위 3개의 문서만 반환하도록 지정합니다.
retriever_generator = vectordb.as_retriever(search_kwargs={"k": 3})
# 'gpt-3.5-turbo-16k'모델을 사용합니다.
# temperature값이 낮은수록 더 일관성 있는 문장이 생성됩니다.
llm_generator = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo-16k')

# 체인타입을 결정합니다.
qa_chain_generator = RetrievalQA.from_chain_type(
    # 언어모델을 할당합니다.
    llm=llm_generator,
    # 체인의 유형을 지정합니다.
    chain_type="stuff",
    retriever=retriever_generator,
    # 응답과 함께 소스 문서를 반환합니다.
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs_generator)

# 언어 모델의 응답을 처리하는 함수를 정의합니다.
def process_llm_response_generator(llm_response_generator):
    # 언어 모델의 응답 결과를 출력합니다.
    return(llm_response_generator['result'])

question = ['올해 금리관련해서 질문 하나만 만들어줄 수 있어?', '올해 경기관련해서 질문 하나만 만들어줄 수 있어?', '은퇴후,건강보험료 관련해서 질문 하나만 만들어줄 수 있어?',\
            '인컴투자 관련해서 질문 하나만 만들어줄 수 있어?','투자원칙 관련해서 질문 하나만 만들어줄 수 있어?', '적립식투자 관련해서 질문 하나만 만들어줄 수 있어?']
answer = []
for i in range(6) :
  result = process_llm_response_generator(llm_response_generator = qa_chain_generator(question[i]))
  answer.append(result)
answer

num1 = random.randrange(0, 3)
num2 = random.randrange(4, 6)

suggested_question_list = []
suggested_question_list.append(answer[num1])
suggested_question_list.append(answer[num2])
suggested_question_list

['은퇴 후 건강보험료를 줄이기 위해 어떤 방법을 사용할 수 있을까요?', '2023년에 명심할 투자 원칙은 무엇인가요?']

## MindMap

In [11]:
# Mind Map

prompt_template_mindmap = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
you are competent financial advisor.

{context}

Question: {question}

Answer in Korean
소주제'들은 주제의 하위주제들로 네가 선정해줘.  '내용'은 문장형식으로, 소주제에 대한 구체적인 너의 설명을 담고 있어야해.  또한 '내용'은 반드시 '~입니다'와 같이 끝나야하고, '소주제'에 대해서 설명하는 어투여야해. let's think step by step.  다음과 같은 형식으로 결과물을 반환해줘. 결과물 이외의 다른 말은 하지마.

'''
small_subject = [소주제1, 소주제2, 소주제3, 소주제4]
contents = [['소주제1'와 관련된 내용1, '소주제1'와 관련된 내용2,'소주제1'와 관련된 내용3],['소주제2'와 관련된 내용1, '소주제2'와 관련된 내용2,'소주제2'와 관련된 내용3],['소주제3'와 관련된 내용1, '소주제3'와 관련된 내용2,'소주제3'와 관련된 내용3],['소주제4'와 관련된 내용1, '소주제4'와 관련된 내용2,'소주제4'와 관련된 내용3]]
'''

"""
# PROMPT에 'context'와 'question'을 입력 변수로하는 프롬프트 템플릿을 저장합니다.
PROMPT_mindmap = PromptTemplate(
    template=prompt_template_mindmap, input_variables=["context", "question"]
)
# 'chain_type_kwargs'변수에 딕셔너리를 저장합니다.
chain_type_kwargs_mindmap = {"prompt": PROMPT_mindmap}
# 'as_retriever'메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 검색 가능한 형태로 변환합니다.
# 유사도가 높은 상위 3개의 문서만 반환하도록 지정합니다.
retriever_mindmap = vectordb.as_retriever(search_kwargs={"k": 3})
# 'gpt-3.5-turbo-16k'모델을 사용합니다.
# temperature값이 낮은수록 더 일관성 있는 문장이 생성됩니다.
llm_mindmap = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo-16k')

# 체인타입을 결정합니다.
qa_chain_mindmap = RetrievalQA.from_chain_type(
    # 언어모델을 할당합니다.
    llm=llm_mindmap,
    # 체인의 유형을 지정합니다.
    chain_type="stuff",
    retriever=retriever_mindmap,
    # 응답과 함께 소스 문서를 반환합니다.
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs_mindmap)

# 언어 모델의 응답을 처리하는 함수를 정의합니다.
def process_llm_response_mindmap(llm_response_mindmap):
    # 언어 모델의 응답 결과를 출력합니다.
    return(llm_response_mindmap['result'])

In [13]:
query = "ETF 투자에 대해서 설명해줄래?"
result = process_llm_response_mindmap(qa_chain_mindmap(query))
print(result)

small_subject = ["상장지수펀드(ETF- Exchange Traded Fund)란?", "ETF 투자 핵심 전략 및 투자 방법", "ETF 투자의 매력과 주의할 점"]
contents = [["상장지수펀드는 특정 지수나 특정 자산의 수익률을 추적하는 펀드를 말합니다. 한국거래소에 상장되어 투자자들이 주식처럼 편리하게 매매할 수 있고, 개별 주식을 선택할 필요가 없어서 펀드 투자의 장점과 주식투자의 장점을 모두 가지고 있는 상품입니다."],
            ["ETF 투자의 핵심 전략 중 하나는 분할 매수 전략입니다. 이는 가치 있는 투자 대상의 경우 가격이 하락할 때 분할로 매수하여 평균 매입가를 낮출 수 있는 장점이 있습니다. 또 다른 전략으로는 코어-위성 전략이 있습니다. 이는 국가 산업 전체 지수를 추적하는 코어 ETF와 유망한 섹터 ETF를 조합하여 초과 수익을 추구하는 전략입니다. 마지막으로 로테이션 전략은 단기적인 주가 변동은 예측할 수 없지만, 거시적인 흐름과 시장 방향성은 예측할 수 있기 때문에 이를 활용하여 탄력적으로 운영하는 전략입니다."],
            ["ETF 투자의 매력은 운용보수가 저렴하고 현금화가 쉽다는 점입니다. 또한, 소액으로 분할 매수가 가능하고 다양한 투자가 가능하다는 점이 매력적입니다. 하지만 주의할 점은 시장 변동성으로 인해 원금 손실이 발생할 수 있다는 점과 해외 지수상품 ETF는 배당소득세가 부과된다는 점입니다. 따라서 ETF에 투자하기 전에 전문가와 충분한 상담을 진행하는 것이 좋습니다."]]




In [17]:
exec(result)

In [18]:
small_subject

['상장지수펀드(ETF- Exchange Traded Fund)란?',
 'ETF 투자 핵심 전략 및 투자 방법',
 'ETF 투자의 매력과 주의할 점']

## Chatbot

In [9]:
from datetime import datetime
date = datetime.today().strftime("%Y/%m/%d")

In [105]:
# 프롬프트를 'prompt_template'변수에 저장합니다.
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
you are competent financial advisor. Today's date is August 18, 2023.

{context}

Question: {question}

Don't answer with short answers, answer with complete sentences.
Your answers should be detailed based on evidence, and don't say unnecessary things. Let's think step by step.
Answer in Korean:"""
# PROMPT에 'context'와 'question'을 입력 변수로하는 프롬프트 템플릿을 저장합니다.
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
# 'chain_type_kwargs'변수에 딕셔너리를 저장합니다.
chain_type_kwargs = {"prompt": PROMPT}

In [106]:
# 'as_retriever'메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 검색 가능한 형태로 변환합니다.
# 유사도가 높은 상위 3개의 문서만 반환하도록 지정합니다.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
# 'gpt-3.5-turbo-16k'모델을 사용합니다.
# temperature값이 낮은수록 더 일관성 있는 문장이 생성됩니다.
llm = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo-16k')

# 체인타입을 결정합니다.
qa_chain = RetrievalQA.from_chain_type(
    # 언어모델을 할당합니다.
    llm=llm,
    # 체인의 유형을 지정합니다.
    chain_type="stuff",
    retriever=retriever,
    # 응답과 함께 소스 문서를 반환합니다.
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

# 언어 모델의 응답을 처리하는 함수를 정의합니다.
def process_llm_response(llm_response):
    # 언어 모델의 응답 결과를 출력합니다.
    return(llm_response['result'])

In [126]:
query = "작년은?"
answer = process_llm_response(llm_response = qa_chain(query))
print(answer)

작년은 2022년이었습니다. 작년 경제는 부진한 모습을 보였으며, 경기 침체와 기업 실적의 둔화가 예상되었습니다. 코스피 지수에 편입된 기업의 주당순이익은 지난해 17% 감소하였고, 올해도 3% 감소할 것으로 예상되었습니다. 그러나 내년에는 경기 여건의 개선으로 인해 주당순이익이 약 30% 증가할 것으로 전망되었습니다. 따라서 작년은 경제와 기업 실적이 부진한 상황이었으며, 이는 올해의 경제와 기업 실적에 영향을 미칠 수 있습니다.
